# Datei in mehrere Dateien trennen

### Funktion:
#### - Die Datei enthält beispielsweise drei Arten von Spalten, die mit F, G und K beginnen.
#### - Für diese Datei werden drei neue Dateien erstellt, die jeweils nur die Spalten mit F, G bzw. K enthalten.
#### - Die Originaldatei bleibt unverändert.

### UseCase:
#### Da die Brabender-Daten, in diesem speziellen Fall die FIBEWE-Dateien, inhomogen verschickt werden, trennt dieses Programm die Zeilen, die mit F, K, G und G mit Ust. oder Vst. beginnen. Anschließend schreibt es diese Zeilen in vier neu erstellte Dateien, die den Namen der Quelldatei und ein Kennzeichen für ihren Inhalt enthalten. Dies soll die spätere Analyse vereinfachen. Zusätzlich werden die Zeilen mit einer ID versehen, sodass ganze Buchungen auch während der Analyse nachvollzogen werden können.

In [3]:
import os
import tkinter as tk
from tkinter import simpledialog

def main():
    # Initialisierung des GUI-Fensters
    root = tk.Tk()
    root.withdraw()  # Versteckt das Hauptfenster

    # Eingabe des Pfades des Quell- und Zielordners
    source_folder_path = simpledialog.askstring("Input", "Was ist der Pfad der Dateien, die bearbeitet werden sollen? Bspw. C:/Benutzer/DeinBenutzername/Dokumente/Zielordner/")
    destination_folder_path = simpledialog.askstring("Input", "Wo sollen die Dateien mit den separierten Zeilen abgelegt werden? C:/Benutzer/DeinBenutzername/Dokumente/Zielordner/")

    if not source_folder_path or not destination_folder_path:
        print("Ungültiger Pfad.")
        return

    source_folder = os.path.abspath(source_folder_path)
    destination_folder = os.path.abspath(destination_folder_path)

    if not os.path.isdir(source_folder):
        print("Quellordner existiert nicht.")
        return

    if not os.path.isdir(destination_folder):
        os.makedirs(destination_folder)

    id_counter = 1

    for source_file_name in os.listdir(source_folder):
        source_file_path = os.path.join(source_folder, source_file_name)
        if os.path.isfile(source_file_path):
            try:
                lines = read_file_with_fallback(source_file_path)
                for line in lines:
                    if line:
                        first_char = line[0]
                        if first_char == 'F':
                            write_to_file(destination_folder, source_file_name, "F", f"ID:{id_counter};{line.strip()}")
                            id_counter += 1
                        elif first_char == 'K':
                            write_to_file(destination_folder, source_file_name, "K", f"ID:{id_counter - 1};{line.strip()}")
                        elif first_char == 'G' and ("Ust." in line or "Vst." in line):
                            write_to_file(destination_folder, source_file_name, "Steuerart", f"ID:{id_counter - 1};G2;{line.strip()}")
                        else:
                            write_to_file(destination_folder, source_file_name, "G", f"ID:{id_counter - 1};{line.strip()}")
            except IOError as e:
                print(f"Fehler beim Verarbeiten der Datei {source_file_path}: {e}")

    print("Verarbeitung der Dateien im Ordner abgeschlossen.")

def read_file_with_fallback(file_path):
    encodings = ['utf-8', 'latin-1', 'cp1252']  # Liste der zu versuchenden Codierungen
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                return f.readlines()
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Keine der Codierungen konnte die Datei lesen: {file_path}")

def write_to_file(destination_folder, source_file_name, marker, content):
    new_file_name = f"{source_file_name}_{marker}.txt"
    new_file_path = os.path.join(destination_folder, new_file_name)

    try:
        with open(new_file_path, 'a', encoding='utf-8') as writer:
            writer.write(content)
            writer.write('\n')
    except IOError as e:
        print(f"Fehler beim Schreiben in die Datei {new_file_path}: {e}")

if __name__ == "__main__":
    main()


Verarbeitung der Dateien im Ordner abgeschlossen.
